In [501]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

df_raw = pd.read_csv('./podatki/bicikelj_train.csv')

display(df_raw.head(5))

,timestamp,PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE,POGAČARJEV TRG-TRŽNICA,KONGRESNI TRG-ŠUBIČEVA ULICA,CANKARJEVA UL.-NAMA,BREG,GRUDNOVO NABREŽJE-KARLOVŠKA C.,MIKLOŠIČEV PARK,BAVARSKI DVOR,TRG OF-KOLODVORSKA UL.,...,POVŠETOVA - KAJUHOVA,SOSESKA NOVO BRDO,TEHNOLOŠKI PARK,VOJKOVA - GASILSKA BRIGADA,GERBIČEVA - ŠPORTNI PARK SVOBODA,DOLENJSKA C. - STRELIŠČE,ROŠKA - STRELIŠKA,LEK - VEROVŠKOVA,VOKA - SLOVENČEVA,SUPERNOVA LJUBLJANA - RUDNIK
0,2022-08-02 13:04:00,17,17,20,26,10,10,16,7,12,...,9,7,5,4,2,9,4,8,3,1
1,2022-08-02 13:10:00,18,17,19,25,10,11,15,7,12,...,9,7,5,4,1,9,4,8,2,1
2,2022-08-02 13:14:00,17,18,19,25,9,11,16,7,12,...,9,7,5,4,1,9,4,8,2,1
3,2022-08-02 13:20:00,18,15,20,26,12,11,18,6,13,...,9,7,5,2,1,9,4,8,2,1
4,2022-08-02 13:25:00,20,16,19,23,12,10,17,6,15,...,9,7,5,2,1,9,4,8,2,1


In [502]:
def add_closest_times(df_ts, type, diff):
    df_closest_times = pd.read_csv(f'./generiraj/closest_{diff}h_{type}.csv')
    df_closest_times = df_closest_times.rename(columns={col: col+f'_closest_{diff}h' for col in df_closest_times.columns[1:]})

    # merge the two data frames on the "timestamp" column
    merged_df = pd.merge(df_ts, df_closest_times, on='timestamp', how='outer')
    return merged_df

In [503]:
def add_is_rainy(df):
    df_precipitation = pd.read_csv('./podatki/export.csv')
    df_precipitation = df_precipitation.loc[:, ['date', 'prcp']] 
    # Convert the date column in precipitation_df to datetime type
    # Convert the date column in precipitation_df to datetime type
    df_precipitation['date'] = pd.to_datetime(df_precipitation['date'])

    # Group precipitation data by date and check if precipitation was more than 5
    rainy_dates = df_precipitation.groupby('date').sum()['prcp'] > 5

    # Map the is_rainy values to timestamp_df based on the corresponding date
    df['is_rainy'] = df['timestamp'].dt.date.map(rainy_dates).astype(int)
    #display(df.head(5))
    return df

In [504]:
df = df_raw.copy()
def preprocess_data(df, type):
    
    df = add_closest_times(df, type, 1)
    df = add_closest_times(df, type, 2)
    #df = add_closest_times(df, type, 3)
    #df = add_times_minus(df, "90", suff)
    # Convert 'timestamp' column to timestamptime
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Extract day, hour, minute, and second values
    df['is_august'] = (df['timestamp'].dt.month == 8).astype(int)
    #df['day'] = df['timestamp'].dt.day
    df['hour'] = df['timestamp'].dt.hour
    #df['minute'] = df['timestamp'].dt.minute
    df['dayofweek'] = df['timestamp'].dt.dayofweek
    # df['rush_hour'] = ((df['hour'] >= 7) & (df['hour'] <= 9)) | ((df['hour'] >= 15) & (df['hour'] <= 17))
    #df['is_weekend'] = ((df['dayofweek'] == 5) | (df['dayofweek'] == 6)).astype(int)
    for hour in range(24):
        df[f'hour_{hour}'] = (df['hour'] == hour).astype(int)
    
    one_hot_encoded = pd.get_dummies(df['dayofweek'], prefix='dayofweek')
    df = pd.concat([df, one_hot_encoded], axis=1)
    
    df = add_is_rainy(df)

    #df = df.drop('day', axis=1)
    df = df.drop('hour', axis=1)
    df = df.drop('dayofweek', axis=1)
    df = df.drop('timestamp', axis=1)
    
    if type == "train":
        df = df.dropna()

    return df

    
df = preprocess_data(df, "train")
df.to_csv('./test/test.csv', index=False)


In [505]:
# NORMALIZACIJA
from sklearn.preprocessing import Normalizer

def normalizacija(X):
    normalizer = Normalizer(norm='l1')
    normalizer.fit(X)
    normalized_data = normalizer.transform(X)
    X = pd.DataFrame(normalized_data, columns=X.columns)
    return X

In [506]:
# STANDARDIZACIJA
from sklearn.discriminant_analysis import StandardScaler


def standardizacija(X):
    scaler = StandardScaler()
    # fit the scaler to the data
    scaler.fit(X)
    print(X.columns)
    # transform the data
    df_scaled = scaler.transform(X)
    # convert the scaled data back to a dataframe
    X = pd.DataFrame(df_scaled, columns=X.columns)
    return X

In [507]:
def split_x_y(df):
    X = df.iloc[:, 83:]
    y = df.iloc[:, :83]
    #X = standardizacija(X)
    #X = normalizacija(X)
    return X, y

X, y = split_x_y(df)

display(X.tail(5))
display(y.tail(5))

,PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE_closest_1h,POGAČARJEV TRG-TRŽNICA_closest_1h,KONGRESNI TRG-ŠUBIČEVA ULICA_closest_1h,CANKARJEVA UL.-NAMA_closest_1h,BREG_closest_1h,GRUDNOVO NABREŽJE-KARLOVŠKA C._closest_1h,MIKLOŠIČEV PARK_closest_1h,BAVARSKI DVOR_closest_1h,TRG OF-KOLODVORSKA UL._closest_1h,MASARYKOVA DDC_closest_1h,...,hour_22,hour_23,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,is_rainy
7734,18.0,13.0,5.0,15.0,12.0,6.0,13.0,20.0,17.0,0.0,...,0,0,False,False,False,False,False,True,False,0
7735,20.0,16.0,7.0,14.0,12.0,5.0,13.0,16.0,17.0,1.0,...,0,0,False,False,False,False,False,True,False,0
7736,20.0,17.0,7.0,13.0,12.0,5.0,13.0,16.0,16.0,1.0,...,0,0,False,False,False,False,False,True,False,0
7737,20.0,15.0,7.0,12.0,12.0,5.0,13.0,16.0,17.0,1.0,...,0,0,False,False,False,False,False,True,False,0
7738,20.0,15.0,7.0,13.0,12.0,4.0,13.0,16.0,17.0,1.0,...,0,0,False,False,False,False,False,True,False,0


,PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE,POGAČARJEV TRG-TRŽNICA,KONGRESNI TRG-ŠUBIČEVA ULICA,CANKARJEVA UL.-NAMA,BREG,GRUDNOVO NABREŽJE-KARLOVŠKA C.,MIKLOŠIČEV PARK,BAVARSKI DVOR,TRG OF-KOLODVORSKA UL.,MASARYKOVA DDC,...,POVŠETOVA - KAJUHOVA,SOSESKA NOVO BRDO,TEHNOLOŠKI PARK,VOJKOVA - GASILSKA BRIGADA,GERBIČEVA - ŠPORTNI PARK SVOBODA,DOLENJSKA C. - STRELIŠČE,ROŠKA - STRELIŠKA,LEK - VEROVŠKOVA,VOKA - SLOVENČEVA,SUPERNOVA LJUBLJANA - RUDNIK
7734,18,17,14,7,7,5,10,20,25,4,...,18,3,8,3,2,17,13,13,11,7
7735,20,17,13,7,7,5,11,19,24,4,...,18,2,8,3,2,17,13,13,11,7
7736,19,18,14,5,8,3,10,20,23,5,...,17,3,8,2,2,17,13,13,11,7
7737,19,17,16,4,6,3,9,17,21,5,...,17,3,8,1,1,17,13,13,11,7
7738,20,17,15,3,6,6,9,16,21,5,...,17,3,8,2,1,19,13,13,11,7


In [508]:
from sklearn.model_selection import train_test_split

def split_train_test(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
#X_train, X_test, y_train, y_test = split_train_test(X, y)

In [509]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import cross_validate
from scipy import stats

def cross_validation(model, X, y, n_folds=5):
# Create an instance of Leave-One-Out Cross-Validation
    # Perform cross-validation on your data
    scores = cross_val_score(model, X, y, cv=n_folds, scoring='neg_mean_absolute_error')

    # Compute the average cross-validation score for each output variable
    mean_scores = -np.mean(scores, axis=0).round(3)
    std_scores = np.std(scores, axis=0).round(3)

    print("Cross-validation scores:", mean_scores)
    data = np.array([[mean_scores, std_scores]])
    scores_df = pd.DataFrame(data, columns=['MAE mean','MAE std'])
    display(scores_df)
    
    return scores_df

In [510]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def display_cv_scores(scores, n_folds=5):
    
    # Calculate the mean and standard deviation of the cross-validation scores
    # Convert the negative MSE and MAE scores to positive values
    mse_scores = -scores['test_neg_mean_squared_error']
    mae_scores = -scores['test_neg_mean_absolute_error']
    

    # Calculate the mean and standard deviation of the scores for each metric
    mse_mean = np.mean(mse_scores).round(4)
    mse_std = np.std(mse_scores).round(4)
    mae_mean = np.mean(mae_scores).round(4)
    mae_std = np.std(mae_scores).round(4)
    # r2_mean = np.mean(r2_scores)
    # r2_std = np.std(r2_scores)

    data = np.array([[mse_mean, mse_std, mae_mean, mae_std]])
    scores_df = pd.DataFrame(data, columns=['MSE mean','MSE std', 'MAE mean','MAE std'])
    display(scores_df)
    
    return scores_df

### Linear regression

In [511]:
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import cross_val_score
# from sklearn.multioutput import MultiOutputRegressor

# # Create a multi-output regression model
# model = MultiOutputRegressor(LinearRegression())

# score = cross_validation(model, X, y)

# model.fit(X,y)
    


### Ridge

In [512]:
# from sklearn.linear_model import LinearRegression, Ridge
# from sklearn.model_selection import cross_val_score
# from sklearn.multioutput import MultiOutputRegressor

# # Create a multi-output regression model
# model = MultiOutputRegressor(Ridge())

# score = cross_validation(model, X, y)

# model.fit(X,y)
# print(model.get_params())

### Lasso

In [513]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor

# Create a multi-output regression model
model = MultiOutputRegressor(Lasso())

score = cross_validation(model, X, y)

model.fit(X,y)
print(model.get_params())

Cross-validation scores: 1.658


,MAE mean,MAE std
0,1.658,0.064


{'estimator__alpha': 1.0, 'estimator__copy_X': True, 'estimator__fit_intercept': True, 'estimator__max_iter': 1000, 'estimator__positive': False, 'estimator__precompute': False, 'estimator__random_state': None, 'estimator__selection': 'cyclic', 'estimator__tol': 0.0001, 'estimator__warm_start': False, 'estimator': Lasso(), 'n_jobs': None}


### Predict on final test set

In [514]:
test_df = pd.read_csv('./podatki/bicikelj_test.csv')
X_raw = test_df.iloc[:, 0]

test_df = preprocess_data(test_df, "test")

def predict(df, model):
    
    
    X_test, y_test = split_x_y(df)
    display(X_test.head(5))

    column_names = y.columns

    pred = model.predict(X_test).clip(min=0)
    #pred =  np.round(pred).clip(min=0).astype(int)

    y_pred_df = pd.DataFrame(pred, index=df.index, columns=column_names)
    
    # write the modified DataFrame back to a CSV file
    return y_pred_df


y_pred_df = predict(test_df, model)

final_df = pd.concat([X_raw, y_pred_df], axis=1)

display(final_df.head(10))

final_df.to_csv('./output/bicikelj_test_oddaja.csv', index=False)

,PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE_closest_1h,POGAČARJEV TRG-TRŽNICA_closest_1h,KONGRESNI TRG-ŠUBIČEVA ULICA_closest_1h,CANKARJEVA UL.-NAMA_closest_1h,BREG_closest_1h,GRUDNOVO NABREŽJE-KARLOVŠKA C._closest_1h,MIKLOŠIČEV PARK_closest_1h,BAVARSKI DVOR_closest_1h,TRG OF-KOLODVORSKA UL._closest_1h,MASARYKOVA DDC_closest_1h,...,hour_22,hour_23,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,is_rainy
0,11.0,11.0,0.0,0.0,0.0,13.0,6.0,2.0,2.0,1.0,...,0,1,False,False,False,True,False,False,False,0
1,11.0,11.0,0.0,0.0,0.0,13.0,6.0,2.0,2.0,1.0,...,0,0,False,False,False,False,True,False,False,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,True,False,False,False,False,False,False,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,True,False,False,False,False,False,False,0
4,4.0,4.0,3.0,16.0,19.0,7.0,3.0,3.0,1.0,2.0,...,0,0,False,False,False,True,False,False,False,0


,timestamp,PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE,POGAČARJEV TRG-TRŽNICA,KONGRESNI TRG-ŠUBIČEVA ULICA,CANKARJEVA UL.-NAMA,BREG,GRUDNOVO NABREŽJE-KARLOVŠKA C.,MIKLOŠIČEV PARK,BAVARSKI DVOR,TRG OF-KOLODVORSKA UL.,...,POVŠETOVA - KAJUHOVA,SOSESKA NOVO BRDO,TEHNOLOŠKI PARK,VOJKOVA - GASILSKA BRIGADA,GERBIČEVA - ŠPORTNI PARK SVOBODA,DOLENJSKA C. - STRELIŠČE,ROŠKA - STRELIŠKA,LEK - VEROVŠKOVA,VOKA - SLOVENČEVA,SUPERNOVA LJUBLJANA - RUDNIK
0,2022-08-04 23:24:00,5.565401,7.339788,0.680785,1.904144,0.000000,8.841742,4.752740,2.446948,2.113061,...,5.332989,2.645626,2.498728,3.104528,4.950233,1.784649,4.104503,0.545453,3.775925,8.334371
1,2022-08-05 00:24:00,5.565401,7.339788,0.680785,1.904144,0.000000,8.841742,4.752740,2.446948,2.113061,...,5.332989,2.645626,2.498728,3.104528,4.950233,1.784649,4.104503,0.545453,3.775925,8.334371
2,2022-08-08 04:58:00,0.212910,0.350263,0.000000,0.372398,0.287918,0.015381,0.306528,0.391730,1.908930,...,10.411136,12.494097,5.195138,7.880820,18.485413,6.078499,5.879065,2.322851,8.832248,0.786771
3,2022-08-08 05:58:00,0.212910,0.350263,0.000000,0.372398,0.287918,0.015381,0.306528,0.391730,1.908930,...,10.411136,12.494097,5.195138,7.880820,18.485413,6.078499,5.879065,2.322851,8.832248,0.786771
4,2022-08-11 21:13:00,4.684311,3.916517,3.455107,9.552976,15.674546,6.422037,2.999430,3.089231,1.029934,...,13.847057,11.469931,1.869888,1.932975,8.318397,7.185160,1.366053,6.558252,2.527258,5.031066
5,2022-08-11 22:13:00,4.684311,3.916517,3.455107,9.552976,15.674546,6.422037,2.999430,3.089231,1.029934,...,13.847057,11.469931,1.869888,1.932975,8.318397,7.185160,1.366053,6.558252,2.527258,5.031066
6,2022-08-16 07:10:00,2.489999,1.702928,2.564289,3.038617,0.724196,0.659331,1.360925,2.322274,2.372447,...,13.909845,9.769144,7.851482,11.726201,18.327823,1.484336,12.291129,2.547747,6.057244,5.844404
7,2022-08-16 08:10:00,2.489999,1.702928,2.564289,3.038617,0.724196,0.659331,1.360925,2.322274,2.372447,...,13.909845,9.769144,7.851482,11.726201,18.327823,1.484336,12.291129,2.547747,6.057244,5.844404
8,2022-08-18 17:30:00,1.697052,0.000000,0.833537,1.689634,3.877193,5.985353,6.280068,1.647139,19.128176,...,9.237335,7.963097,6.433638,1.074249,11.197626,7.104312,2.330063,0.158609,3.932588,4.134477
9,2022-08-18 18:27:00,1.653124,0.000000,0.798209,1.560429,3.879457,5.992316,6.269395,1.508952,19.131501,...,9.237335,7.963097,6.419564,1.074249,11.197626,7.107040,2.330063,0.166188,3.932588,4.134477
